# Question 1

It is quite clear that $var(\bar{X})=var(F)$.\
We need to find $var(\bar{Y})$.\
We know:
$$
P(X_{t+1}|X_{t}=x)=F\\
var(X)=var(F)
$$
Both the estimators have the same variance and are equal.


# Question 2

## Question 2 (a)
The joint posterior distribution for $\theta=(\mu, \nu)$ is:
$$
P(\theta|D) \propto P(\theta)P(D|\theta)\\
P(\theta|D) \propto \nu^{a_{0}-1}e^{\frac{-\nu}{b_{0}}}I(\{\mu > 2\})(1+\frac{\mu^{2}}{\nu})^{-\frac{\nu + 1}{2}}\prod_{i=1}^{n}e^{-\frac{(y_{i}-\mu)^{2}}{2}}\\
P(\theta|D)=f(\theta)=c\bar{f}(\theta)
$$
$c\bar{f}(\theta)$ is the posterior distribution for $\theta$.


In [1]:
using Distributions
function sampleY(mu,n)
    f=Normal(mu,1)
    Y=rand(f,n)
    return Y
end

function sampleTheta() # Theta is [mu,nu]
    f=Normal(0,1)
    Y=rand(f,2)
    while(Y[2]<2)
        Y=rand(f,2)
    end
    return Y
end

function samplev(a,b)
    vfunc=truncated(Gamma(a,b),lower=2)
    v=rand(vfunc,1)
    return v[1]
end


function samplemu(a,b,v)
    mufunc=TDist(v)
    mu=rand(mufunc,1)
    return mu[1]
end


# function norm(x,mu)
#     return exp(-(x-mu)^2/2)
# end

function mini(a,b)
    if(a<b)
        b=a
    end
    return b
end

function post(mu,nu,a,b,Y,n)
    prod=1
    if(mu<2)
        return 0
    end
    prod=prod*(nu^(a-1))*exp(-nu/b)
    prod=prod*(1+mu^2/nu)^(-(nu+1)/2)
    i=1
    while(i<=n)
        prod=prod*exp(-0.5*(Y[i]-mu)^2)
        i=i+1
    end
    return prod
end


function MH(a,b,Y,n,theta0,num)
    S=[]
    mu0=theta0[1][1]
    nu0=theta0[1][2]
    append!(S,theta0)
    num=num-1
    while(num>0)
        u=rand(Uniform(0,1))
        theta=sampleTheta()
        mu=theta[1]
        nu=theta[2]
        if(u<mini(1,post(mu,nu,a,b,Y,n)/post(mu0,nu0,a,b,Y,n)))
            mu0=mu
            nu0=nu
        end
        theta1=[[mu,nu]]
        append!(S,theta1)
        num=num-1
    end
    return S
end   

MH (generic function with 1 method)

In [2]:
post(3,1,1,1,[1,2],2)

0.00301973834223185

In [3]:
nu=samplev(2,0.1)
mu=samplemu(2,0.1,nu)
theta=[[mu,nu]]
Y=sampleY(mu,100)

Z=MH(2,0.1,Y,100,theta,100000)
println(Z[100000])
println("This is Mu,Nu")

[-0.09945484831349852, 2.1393718145264415]
This is Mu,Nu


# Question 3

## Question 3 (a)

$$
P(\beta,\sigma^{2})=\frac{b^{a}}{\Gamma{a}}(\frac{1}{\sigma^{2}})^{-(a+1)}e^{-\frac{b}{\sigma^{2}}}\frac{1}{\sigma\sqrt{2\pi}}e^{\frac{-(\beta-\mu)^{2}}{2\sigma^{2}}}\\
P(\beta,\sigma^{2})=\frac{b^{a}}{\sqrt{2\pi}\Gamma{a}}\sigma^{2a+2}(\frac{e^{-\frac{(\beta-\mu)^2+2b}{2\sigma^{2}}}}{\sigma})
$$
This is the joint prior distribution for $(\beta,\sigma^{2})$.\
Given that $y_{1},y_{2},y_{3},...,y_{n} \sim N(X\beta,\sigma^{2}I_{n})$, we now can calculate the posterior distribution using Bayesian methods.

The joint posterior distribution for $\theta=(\beta, \sigma^{2})$ is:
$$
P(\theta|D) \propto P(\theta)P(D|\theta)\\
P(\theta|D) \propto \sigma^{2a+2}(\frac{e^{-\frac{(\beta-\mu)^2+2b}{2\sigma^{2}}}}{\sigma})\frac{e^{-\frac{(X\beta-Y)^2}{2\sigma^{2}}}}{\sigma}\\
P(\theta|D)=f(\theta)=c\bar{f}(\theta)
$$

In [4]:
using PyCall
py"""
from sklearn import datasets
x, y, coef = datasets.make_regression(n_samples=100,
                                      n_features=2,#number of features
                                      noise=1,#bias and standard deviation of the guassian noise
                                      coef=True,#true coefficient used to generated the data
                                      random_state=0)
"""
x=py"x"
y=py"y"
c=py"coef"
# Generate Dataset

2-element Vector{Float64}:
 29.214752679254886
 96.19363785472291

In [5]:
using Distributions

function samplesig(a,b)
    g=InverseGamma(a,b)
    sig=rand(g)
    return sig[1]
end

function sampleBeta(mu,sigma,n)
    N=Normal(mu,sigma)
    beta=rand(N,n)
    return beta
end

function sampleTheta(n)
    N=Normal()
    beta=rand(N,n)
    sigma=abs(rand(N))
    theta=[beta,sigma]
    return theta
end


function multiNorm(beta,sigma)
    l=length(beta)
    i=1
    Y=[]
    while(i<=l)
        x=Normal(beta[i],sigma)
        y=rand(x)
        append!(Y,y)
        i=i+1
    end
    return Y
end

function pdfNorm(X,beta,Y,sigma,a,b,mu)
    xb=X*beta
    l1=length(xb)
    l=length(beta)
    prod=Float64(sigma^(2*a+2))
    i=1
    sum=0
    while(i<l)
        prod=prod/sigma
        sum=sum-((beta[i]-mu)^2)/(2*sigma^2)
        i=i+1
    end
    i=1
    while(i<l1)
        prod=prod/sigma
        sum=sum-(((xb[i]-Y[i])^2)/(2*sigma^2))
        i=i+1
    end
    prod=prod*exp(-b/(sigma^2))
    sum=sum+log(prod)
    return sum
end


function sampleY(X,beta,sigma)
    Y=multiNorm(X*beta,sigma)
    return Y 
end


function LinearReg(n,a,b,Y,X,mu,sigma,beta,num)
    S=[]
    beta0=beta
    sigma0=sigma
    theta0=[[beta0,sigma0]]
    append!(S,theta0)
    num=num-1
    while(num>0)
        u=rand(Uniform(0,1))
        theta=sampleTheta(n)
        sig=theta[2]
        bet=theta[1]
        if(log(u)<mini(0,pdfNorm(X,bet,Y,sig,a,b,mu)-pdfNorm(X,beta0,Y,sigma0,a,b,mu)))
            beta0=bet
            sigma0=sig
        end
        theta1=[[beta0,sigma0]]
        append!(S,theta1)
        num=num-1
    end
    return S
end   


LinearReg (generic function with 1 method)

In [6]:
n=2
mu=1
a=2
b=3
sigma=sqrt(samplesig(a,b))
beta=sampleBeta(mu,sigma,n)
println("Linear Reg:")
num_epoch=10000
res=LinearReg(n,a,b,y,x,mu,sigma,beta,num_epoch)

Linear Reg:


10000-element Vector{Any}:
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[0.9682289289964102, 1.7221699511579738], 1.1567418897457777]
 Any[[1.8920519079818947, 0.16412213085255697], 1.1991640070924054]
 Any[[1.8920519079818947, 0.16412213085255697], 1.1991640070924054]
 Any[[1.8920519079818947, 0.16412213085255697], 1.1991640070924054]
 Any[[1.8920519079818947, 0.16412213085255697], 1.1991640070924054]
 ⋮
 Any[[1.0315382784831375, 0.773104203150817], 3.8420180791760803]
 Any[[1.0315382784831375, 0.7

In [7]:
pdfNorm(x,beta,y,sigma,a,b,mu)

-369478.24429685867

In [8]:
betaFin=res[num_epoch][1]
sigmaFin=res[num_epoch][2]

3.8420180791760803

In [9]:
println("Final Beta:",betaFin)
println("Final Sigma:",sigmaFin)

Final Beta:[1.0315382784831375, 0.773104203150817]
Final Sigma:3.8420180791760803


In [10]:
x

100×2 Matrix{Float64}:
  1.05445    -1.07075
 -0.362741   -0.634322
 -0.854096    0.313068
  1.32639     0.298238
 -0.461585   -1.31591
  1.94362    -1.17312
 -1.60206     0.625231
 -0.401781    0.177426
 -0.977278    1.86756
  0.378163    0.154947
 -0.302303   -0.387327
 -0.747455   -0.413619
 -0.187184    0.0457585
  ⋮          
  0.428332   -0.0281822
  0.653619   -2.55299
  0.944479    2.38314
  0.0318306  -0.674333
  1.92953    -0.498032
 -1.45437     2.26975
  0.462782   -1.6302
  0.823504    0.771791
  0.976639    0.208275
 -1.34776     1.88315
  0.0521651  -1.10438
 -1.72628    -0.813146

In [11]:
y

100-element Vector{Float64}:
  -72.50305646012106
  -71.85306494343101
    5.0647749388690375
   66.93183449869574
 -140.7194095496918
  -55.922642586225884
   15.643486286718097
    4.786473915337476
  152.19592501357135
   25.542831343549583
  -46.1323328491405
  -61.94355031734867
   -1.533270738727035
    ⋮
   10.034839631641375
 -225.5758932706944
  258.79201235994236
  -63.41269884142541
    8.35270787992977
  176.63334879245508
 -142.87859023362495
   97.90036194227108
   49.71932728788642
  142.29373341244977
 -105.77067693938837
 -129.14573814288124